# Lab 1: Introduction to feedforward neural networks

In today's lab, we'll build a simple feedforward neural network, and train it to compute the XOR function. We'll do this in core Python (plus NumPy) first: it's important to see every detail at least once. Then we'll repeat the exercise in TensorFlow.

In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

### Quick demo of linear algebra operations in NumPy

In [3]:
vector = np.array([1,-2,3])

In [4]:
vector

array([ 1, -2,  3])

In [5]:
matrix = np.array([[1,2,3], [4,5,6], [7,8,9], [10,11,12]])

In [6]:
matrix

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [7]:
# matrix acting on a vector
matrix @ vector

array([ 6, 12, 18, 24])

In [8]:
# dot product of two vectors
vector @ vector

14

### XOR 'truth table' as NumPy array

We want our neural net to learn to compute this function. (Notice that linear regression can't solve this problem -- at least, not unless we include an interaction term. The XOR function is non-linear.)

In [9]:
xor_table = np.array([[0,0,0], [0,1,1], [1,0,1], [1,1,0]])

In [10]:
# inputs in cols 1 and 2, output in col 3
xor_table

array([[0, 0, 0],
       [0, 1, 1],
       [1, 0, 1],
       [1, 1, 0]])

### Data for training a neural network to compute XOR

We'll train a neural network to compute XOR by showing it lots of slightly noisy examples of correctly evaluated XORs. We'll add noise to the inputs (the 'feature vectors' in machine learning parlance), but leave the desired outputs (the 'labels') untouched. Our neural net should eventually learn that the output should be roughly 1 if the feature vector is close to (0,1) or (1,0), and roughly zero otherwise.

In [11]:
'''
Generate slightly noisy feature vectors, and noise-free labels.
'''
n = 100 # total number of cases
noise_level = 0.2 # amplitude of Gaussian noise
data = xor_table[np.random.choice(range(4), size = n), :] # sample with replacement from XOR table
noise = np.concatenate((noise_level * np.random.randn(n,2), np.zeros((n,1))), axis=1) # noise to inject into data
data = data + noise
features = data[:,:2]
labels = data[:,2]

Let's inspect what we've assembled so far by looking at the first few rows of a few of these objects.

In [12]:
data[:10]

array([[ 0.83838459,  1.00932889,  0.        ],
       [-0.15055696,  0.24436154,  0.        ],
       [-0.12628471,  0.03940612,  0.        ],
       [ 0.28210009,  0.1937349 ,  0.        ],
       [ 0.80609337,  1.06311941,  0.        ],
       [ 0.08992244,  0.74923709,  1.        ],
       [ 0.89738992,  0.81574618,  0.        ],
       [ 0.90330091,  0.89253292,  0.        ],
       [ 1.23942807,  0.90476843,  0.        ],
       [ 0.90138015,  0.15160295,  1.        ]])

In [13]:
features[:10]

array([[ 0.83838459,  1.00932889],
       [-0.15055696,  0.24436154],
       [-0.12628471,  0.03940612],
       [ 0.28210009,  0.1937349 ],
       [ 0.80609337,  1.06311941],
       [ 0.08992244,  0.74923709],
       [ 0.89738992,  0.81574618],
       [ 0.90330091,  0.89253292],
       [ 1.23942807,  0.90476843],
       [ 0.90138015,  0.15160295]])

In [14]:
labels[:10]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 1.])

### Implementing a simple neural network

**Exercise**. Implement a neural network with two inputs, a single hidden layer of two ReLU neurons, and a single linear output neuron. Do this by writing a `relu(x)` function, and a  `forward_pass(feature_vector, weights)` function. The latter should take a feature vector (e.g. (1,0)) and a matrix of weights, and compute the output of the network with those weights when fed that input vector.

**Solution**

In [15]:
def relu(x):
    return np.maximum(0,x)

In [16]:
def forward_pass(feature_vector, weights):
    '''
    Implements feed-forward NN with two inputs, two hidden-layer units (ReLUs),
    and a single output unit (linear).
    feature_vector is the input: (x1, x2).
    First two rows of weights are 1st-layer biases and weights: [b1, a11, a12], [b2, a21, a22].
    (NB that a_ij is weight for connection *to* ith hidden layer neuron *from* jth input.)
    Final row of weights are 2nd-layer biases and weights: [b, w1, w2].
    '''
    feature_vector = np.append(1, feature_vector) # add dummy input to play nicely with biases
    hidden_layer_output = relu(weights[:2] @ feature_vector)
    hidden_layer_output = np.append(1, hidden_layer_output) # add dummy again
    return (weights[2:] @ hidden_layer_output)[0]

### Finding good weights

The remaining design task is to pick a set of weights that will do the desired job (i.e. compute  XOR). We *could* do this manually -- through some combination of cunning and trial and error.

**Exercise**. Have a go!

**Solution**

Here is one possibility. (Draw a diagram and think through how it works.)

In [17]:
weights_guess = np.array([[0,1,-1], [0,-1,1], [0,1,1]])

In [18]:
for row in xor_table[:,:-1]:
    print(row, forward_pass(row, weights_guess))

[0 0] 0
[0 1] 1
[1 0] 1
[1 1] 0


Picking weights 'manually' won't work for bigger, more complicated problems; and in any case the whole point of deep learning is for networks to learn the right weights algorithmically -- from the data. This process is known as **training**. The standard approach is to initialise weights to random values and then hunt for better choices by doing gradient descent (or something similar). For that we need a loss function. We'll use a simple one for now: mean square error.

In [19]:
def loss(feature_vectors, labels, weights):
    predictions = [forward_pass(vect, weights) for vect in feature_vectors]
    errors = predictions - labels
    return (errors**2).mean()

Not surprisingly, randomly assigned weights do not do a good job computing XOR.

In [20]:
weights = np.random.uniform(-1.2, 1.2, size = (3,3)) # initialize weights

In [21]:
weights

array([[-0.79646998, -0.13308253,  0.09449428],
       [ 0.65721397, -0.0535835 , -0.91239527],
       [ 1.11872704,  0.9829142 , -0.86812605]])

In [22]:
loss(features, labels, weights)

0.451417614125789

To apply gradient descent on the loss function, we need to be able to calculate the gradient of the loss function with respect to each component of the weights matrix in turn. The natural object to store this gradient in is a matrix with the same shape as the weights matrix.

The function below computes each gradient component by testing the effect on the loss of a small change in the corresponding weight. (In a deep network, it would be very inefficient to run through this procedure separately for each weight component -- but we won't worry about that now.)

In [23]:
def loss_gradient(feature_vectors, labels, weights, delta = 0.01):
    current_loss = loss(feature_vectors, labels, weights)
    gradient = np.empty(weights.shape) # initialise empty matrix to hold gradient information
    for i in range(weights.shape[0]):
        for j in range(weights.shape[1]):
            weights[i,j] += delta # perturb weight
            perturbed_loss = loss(feature_vectors, labels, weights)
            weights[i,j] -= delta # restore weight
            gradient[i,j] = (perturbed_loss - current_loss) / delta
    return gradient

In [24]:
loss_gradient(features, labels, weights)

array([[ 0.        ,  0.        ,  0.        ],
       [-0.16875602,  0.17938838, -0.03970261],
       [ 0.74800735,  0.        ,  0.09893411]])

**Q.** If we are going to change the weights by taking a small step in 'weights space', what direction should we step in?

**A.** The direction in which the loss is *decreasing* most quickly. That's in the direction `-loss_gradient`.

Let's take just one step and confirm that the loss has decreased.

In [25]:
learning_rate = 0.1 # controls sizes of steps
weights = weights - learning_rate * loss_gradient(features, labels, weights)

In [26]:
weights

array([[-0.79646998, -0.13308253,  0.09449428],
       [ 0.67408957, -0.07152234, -0.90842501],
       [ 1.0439263 ,  0.9829142 , -0.87801946]])

In [27]:
loss(features, labels, weights)

0.3960301957218067

So the loss has indeed decreased. That's not surprising, but it wasn't a sure thing: if we'd picked too large a value for `learning_rate`, the loss might have increased. (Why?)

Of course, we can *iterate* the above procedure. The hope is that by doing so, we'll converge on a really good set of weights.

**Exercise**. Try this. Write code to perform 100 further rounds of gradient descent. (And use a smaller learning rate, just to be on the safe side.)

**Solution**

In [28]:
learning_rate = 0.01
print('Initial loss: ', loss(features, labels, weights))
for i in range(100):
    weights = weights - learning_rate * loss_gradient(features, labels, weights)
print('Final loss: ', loss(features, labels, weights))

Initial loss:  0.3960301957218067
Final loss:  0.23310575448856008


In addition to the loss, we can compute the accuracy: the fraction of cases the network is getting right. We'll say it gets a case right if the output rounded to the nearest integer matches the ground-truth label (i.e., the desired XOR value).

In [29]:
best_guesses = np.array([round(forward_pass(vect, weights)) for vect in features])
print('Accuracy: ', sum(best_guesses == labels) / n)

Accuracy:  0.66


**Discussion question**. Why would accuracy be a poor choice of loss function?

We can also check the network's output for all four possible 'pristine' (non-noise-corrupted) cases.

In [30]:
for row in xor_table[:,:-1]:
    print(row, forward_pass(row, weights))

[0 0] 0.21674630030615893
[0 1] 0.7439746954638509
[1 0] 0.48623895139901346
[1 1] 0.7439746954638509


If you re-run the above training process many times (starting from the top, with random initial weights), you will *sometimes* converge to a good solution; but rather often you will converge to a poor solution.

**Exercise**. Demonstrate this. (Reuse the code you already have.)

So getting lucky with the initial weights seems to be important. We could keep picking new sets of initial weights and running gradient descent until we end up somewhere good. But could we instead *revise the architecture* of the network to make it more likely that a random initial set of weights will lead us (via gradient descent) to a good solution?

Yes. The trick is to use a bigger network, with more neurons than are strictly necessary for computing XOR. You can think of this as being a bit like a parallel version of the 'pick random weights, train, and start again if you get stuck' strategy. With a larger network, the neurons may not all end up (post-training) doing a useful job; but there's a better chance that *some subset* of the initial weights is such that gradient descent will take us to a good solution.

We'll eventually demonstrate this point, but not with our home-brew neural network code. It's time to introduce TensorFlow.

### First glimpse of TensorFlow

TensorFlow lets you design, train, and run neural networks using a high-level API. Fast C code runs under the hood. Here's a reimplementation of our simple XOR network.

In [31]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [32]:
model = Sequential()
model.add(Dense(2, input_shape=(2,), activation='relu')) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['binary_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 6         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [33]:
history = model.fit(features, labels, epochs=100, verbose=0)
history.history['loss'][-1], history.history['binary_accuracy'][-1]

(0.3719959378242493, 0.57)

And here's an implementation of a lager network, with two hidden layers (of 8 and 4 neurons respectively).

In [34]:
model2 = Sequential()
model2.add(Dense(8, input_shape=(2,), activation='relu')) 
model2.add(Dense(4, activation='relu'))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam', 
              metrics=['binary_accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 8)                 24        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


**Exercise**. Why does the above network have 65 trainable parameters? Make sure you can account for all of them (and that you understand why there are not more).

In [35]:
history = model2.fit(features, labels, epochs=50, verbose=0, validation_split=0.25)
history.history['loss'][-1], history.history['binary_accuracy'][-1]

(0.12737502952416738, 0.8666667)

**Exercise**. Re-run the above two training processes several times each. You should find that the larger network is less likely to get stuck at a bad local optimum.

The XOR problem is a pretty trivial, of course. But it's allowed us to build some intuition for why a bigger network (or what a statistician would call an *over-parameterised model*) might be easier to train. In this course, we will encounter networks with tens of thousands of trainable parameters.